## Регистрация пользователей 

In [274]:
!pip3 install requests
!pip3 install pandas
!pip3 install openpyxl

You should consider upgrading via the '/Users/syper_olao/Desktop/program/programs/work/selenium/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/syper_olao/Desktop/program/programs/work/selenium/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/syper_olao/Desktop/program/programs/work/selenium/.venv/bin/python3 -m pip install --upgrade pip' command.


### Connection to environment

In [275]:
import os
from pathlib import Path
from dotenv import load_dotenv, dotenv_values

dotenv_path = os.path.join(Path(os.path.abspath('')), ".env")
print(dotenv_path)
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    
config_env = os.environ if not os.path.exists(dotenv_path) else dotenv_values(".env")

/Users/syper_olao/Desktop/program/programs/work/selenium/register/.env


In [276]:
# import pandas as pd

# csv_data = pd.read_csv("users.csv", sep=';', on_bad_lines='skip')
# csv_data

In [277]:
from openpyxl import load_workbook

wb = load_workbook("users.xlsx")

In [278]:
sheet_names = wb.sheetnames

In [279]:
import string
import secrets


def generate_password():
    alphabet = string.ascii_letters + string.digits
    return "".join(secrets.choice(alphabet) for _ in range(10))

In [280]:
API_ENDPOINT = "http://localhost:3000/apiv"

sheet = wb[sheet_names[0]]

In [281]:
from typing import Any


class User:
    accessToken: str | None
    refreshToken: str | None
    me: Any

In [282]:
import requests
import json
from requests.exceptions import RequestException


class BaseApi:
    def __init__(self, base_url, access):
        self.base_url = base_url
        self.session = requests.Session()
        self.access = access

    def _get_access_token(self):
        # Реализуйте функцию чтения токена доступа
        return self.access  # Замените на ваш метод чтения токена

    def _refresh_token(self, id, refresh_token):
        # Реализуйте функцию обновления токена
        response = self.session.get(f"{self.base_url}/auth/{id}/{refresh_token}")
        response.raise_for_status()
        return response.json()

    def request(self, method, url, **kwargs):
        access_token = self._get_access_token()
        headers = kwargs.get("headers", {})
        if access_token:
            headers["Authorization"] = f"Bearer {access_token}"
        kwargs["headers"] = headers

        if "json" in kwargs:
            kwargs["data"] = json.dumps(kwargs["json"])
            del kwargs["json"]

        try:
            response = self.session.request(method, f"{self.base_url}{url}", **kwargs)
            print(response.status_code)
            response.raise_for_status()
            return response.json()
        except RequestException as error:
            print(str(error))
            if response.status_code == 401:
                raise ApiError(error.response.json().get("message", str(error)))


class ApiError(Exception):
    def __init__(self, message):
        super().__init__(message)

In [283]:
def login_to_app(login, password):
    try:
        r = requests.post(
            url=f"{API_ENDPOINT}/auth",
            data=json.dumps({"login": login, "password": password}),
            headers={
                "Access-Control-Allow-Origin": "*",
                "Cache-Control": "no-cache",
                "Content-Type": "application/json",
            },
        )
        print(r)
        print(r.status_code)
        if r.status_code == 200:
            return r.json()
        else:
            return None
    except Exception as err:
        print(err)

In [284]:
login_response = login_to_app(config_env['LOGIN'], config_env['PASSWORD'])

<Response [200]>
200


In [285]:
api = BaseApi(base_url=API_ENDPOINT, access=login_response.get('accessToken'))

In [286]:
def user_register(user):
    try:
        api.request(
            'POST',
            url=f"/registration",
            json=user,
            headers={
                "Access-Control-Allow-Origin": "*",
                "Content-Type": "application/json",
            },
        )
    except Exception as err:
        print(err)

In [287]:
data_list = []

In [288]:
for row in sheet.iter_rows(2, values_only=True):
    if row is None or all(value is None for value in row):
        continue

    email = row[4]
    data = {
        "password": generate_password(),
        "name": row[1],
        "email": email,
        "login": email.split("@")[0],
        "roleId": 1,
        "gender": 2,
    }

    data_list.append(data)

    user_register(data)

201
201


In [289]:
import csv

with open("data.csv", "w", newline="") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=data.keys())
    if csv_file.tell() == 0:
        writer.writeheader()
    writer.writerows(data_list)